In [3]:
import numpy as np
import urllib
import librosa
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from microphone import record_audio
from IPython.display import Audio
from pathlib import Path
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion 
from scipy.ndimage.morphology import iterate_structure
import pickle
from collections import Counter

In [4]:
def audio_to_digital(*file_path_or_music):
    if(len(file_path_or_music)!= 0):
        if(".mp3" in file_path_or_music[0]):
            #analyze file input if it is there
            from pathlib import Path
            local_song_path = Path(file_path_or_music[0])
            frames, sample_rate = librosa.load(local_song_path, sr=44100, mono=True, duration=5) #Seconds
            audio_data = np.hstack([np.frombuffer(i, np.int16) for i in frames])

        else:
            #analyze file input if it is there
            with open(file_path_or_music[0], 'r') as R:
            # each sample is written to a single line in the text file
            # this reads them in as a single integer-valued numpy array
                audio_data = np.asarray([int(i) for i in R])
    else:
        #or else we are recording what they say
        from microphone import record_audio
        listen_time = 5  # seconds
        frames, sample_rate = record_audio(listen_time)
        # read in the recorded audio, saved as a numpy array of 16-bit integers
        audio_data = np.hstack([np.frombuffer(i, np.int16) for i in frames])
        Checking=True
    return audio_data, Checking

In [5]:
def digitalToSpecto(audio):
    S, freqs, times = mlab.specgram(audio, NFFT=4096, Fs=44100,
                                  window=mlab.window_hanning,
                                  noverlap=4096 // 2)
    return(S)

In [6]:
def spectrogram_to_peaks(arr):
    """ 
    Creates a boolean array showing peaks, given data from a spectrogram.
    
    Parameters:
        arr: The array produced by the spectrogram from digital_to_spectrogram with shape (N,M)
        
    Returns:
        peaks: A boolean array with shape (N,M). Peaks in the data are where peaks == True.
    """

    # Creating the histogram
    arr_flattened = np.log(arr.flatten() + 1e-20)
    
    N = arr_flattened.size # Number of elements in the array
    cnt, bin_edges = np.histogram(arr_flattened, bins=N//200, density=True)
    bin_width = np.diff(bin_edges) 
    
    ## print(np.sum(cnt*bin_width)) # check that summation = 1
    
    # Creating the cumulative distribution
    cumulative_distr = np.cumsum(cnt*bin_width)
    
    # Defining the cutoff
    frac_cut = 0.9
    bin_index_of_cutoff = np.searchsorted(cumulative_distr, frac_cut)
    
    # given the bin-index, we want the associated log-amplitude value for that bin
    cutoff_log_amplitude = bin_edges[bin_index_of_cutoff]
    
    # Defining the footprint
    fp = generate_binary_structure(rank=2,connectivity=2)
    
    peaks = ((arr > cutoff_log_amplitude) & (arr == maximum_filter(arr, footprint=fp)))
    freq, time = np.where(peaks)
    print(freq)
    return peaks

In [7]:
def peaks_to_dic_newsongs(local_peaks):
    with open("songs.pkl", mode="rb") as opened_file:
        song_dic = pickle.load(opened_file)
    freq, time= np.where(local_peaks)
    song_name=input("Song Name= ")
    for i in range(len(local_peaks)-15):
        j=1
        while j<16:
            finger=(freq[i], freq[i+j], time[j]-time[i])
            j+=1
            song_dir=(song_name, time[i])
            song_dic.update([(finger, song_dir)])
    print(song_dic)
    # open and save to a pkl file, mode = wb for binary storage w/ numpy array
    with open("songs.pkl", mode="wb") as opened_file:
        pickle.dump(song_dic, opened_file)
    print("Done")
    return "Done"

#ideas for optimization:::::::::::::::::::::
# vstack with tuple inside it (the fingerprint)
# convert that into a list of tuples using a zip function and the map function and slicing

In [8]:
def peaks_to_dic_findsong(local_peaks):
    with open("songs.pkl", mode="rb") as opened_file:
        song_dic2 = pickle.load(opened_file)
    freq, time= np.where(local_peaks)
    songList = []
    fingerList=[]
    for i in range(len(local_peaks)-15):
        j=1
        for j in range(16):
            finger=(freq[i], freq[i+j], time[i+j]-time[i])
            fingerList.append(finger)
            if finger in song_dic2:
                song_guess=song_dic2[finger]
                songList.append((song_guess[0], (song_guess[1]-finger[2])))
                Counter_songs= Counter(songList)
    song=Counter_songs.most_common(1)
    with open("songs.pkl", mode="wb") as opened_file:
        pickle.dump(song_dic2, opened_file)
    return song

In [9]:
def final_function(song):
    Checking=False
    audio=audio_to_digital(song)
    arr=digitalToSpecto(audio)
    local_peaks=spectrogram_to_peaks(arr)
    if Checking==True:
        Checking=False
        
    else:
        peaks_to_dic_newsongs(local_peaks)